In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

# sys.path.append("../response/")
# from feature import Feature

In [16]:
corpus_path = "../../corpus/SNLI/"
data_name = "dev.tsv"
data_name = "train_wo_filtering.tsv"

- data
['train_w_filtering.tsv', 'README.md', 'train_wo_filtering.tsv', 'dev.tsv']

In [17]:
df = pd.read_csv(corpus_path+data_name, delimiter='\t', names=("label", "pre", "hypo"))

In [18]:
df

,label,pre,hypo
0,label,pre,hypo
1,neutral,ガレージ で 、 壁 に ナイフ を 投げる 男 。,男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。
2,contradiction,茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。,女性 が 畑 で 踊って い ます 。
3,contradiction,ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。,黒人 は デスクトップ コンピューター を 使用 し ます 。
4,entailment,海 の 波 に 倒れる 男,海 に 転がる 男 。
...,...,...,...
548010,entailment,犬 が パドリング プール に 飛び 込む と 、 ホース で 潮 吹き さ れ ます 。,犬 が ジャンプ し ます 。
548011,entailment,ブルゴーニュ の シャツ と 黒い ズボン を 着た 男 が 、 楽器 を 持って いる 人...,男 は 人形 と 楽器 を 使って 仕事 を して い ます 。
548012,contradiction,口 ひげ 、 フェイス ペイント 、 それ に 合った 赤い 帽子 と シャツ を 着た 男...,男 は 仕事 を 失った 後 、 口 ひげ を 剃り ます 。
548013,neutral,マヤ 遺跡 を 訪れて いる 間 、 ガイド は より 多く の アトラクション の 方向 ...,人々 は もっと やる こと を 探して いる 観光 客 です 。


In [19]:
df['label'].value_counts()

contradiction    182964
entailment       182583
neutral          182467
label                 1
Name: label, dtype: int64

In [20]:
new_name = "dev_sentence.csv"
new_name = "train_sentence.csv"

In [21]:
for i, (p, h) in  enumerate(zip( df.pre, df.hypo )) :
    p_ = p.replace(" ", "")
    try:
        df.iat[i, 1] = p_
        df.iat[i, 2] = h.replace(" ", "")
    except Exception as e:
        print()
        print(p, h)


空 に 浮かぶ 黒い シャツ と ブルー ジーンズ を 身 に 着けて いる 男 が 、 「 踏み台 」 と いう 言葉 の ある 建物 の 前 で 両手 に 合板 を 持って い ます 。 nan


In [22]:
df

,label,pre,hypo
0,label,pre,hypo
1,neutral,ガレージで、壁にナイフを投げる男。,男は魔法のショーのためにナイフを投げる行為を練習しています。
2,contradiction,茶色のドレスを着た女性がベンチに座っています。,女性が畑で踊っています。
3,contradiction,ラップトップコンピューターを使用して机に座っている若い白人男。,黒人はデスクトップコンピューターを使用します。
4,entailment,海の波に倒れる男,海に転がる男。
...,...,...,...
548010,entailment,犬がパドリングプールに飛び込むと、ホースで潮吹きされます。,犬がジャンプします。
548011,entailment,ブルゴーニュのシャツと黒いズボンを着た男が、楽器を持っている人形を操っています。,男は人形と楽器を使って仕事をしています。
548012,contradiction,口ひげ、フェイスペイント、それに合った赤い帽子とシャツを着た男性がバイオリンを弾いています。,男は仕事を失った後、口ひげを剃ります。
548013,neutral,マヤ遺跡を訪れている間、ガイドはより多くのアトラクションの方向を指し示します。,人々はもっとやることを探している観光客です。


In [23]:
df.dropna(how="any")

,label,pre,hypo
0,label,pre,hypo
1,neutral,ガレージで、壁にナイフを投げる男。,男は魔法のショーのためにナイフを投げる行為を練習しています。
2,contradiction,茶色のドレスを着た女性がベンチに座っています。,女性が畑で踊っています。
3,contradiction,ラップトップコンピューターを使用して机に座っている若い白人男。,黒人はデスクトップコンピューターを使用します。
4,entailment,海の波に倒れる男,海に転がる男。
...,...,...,...
548010,entailment,犬がパドリングプールに飛び込むと、ホースで潮吹きされます。,犬がジャンプします。
548011,entailment,ブルゴーニュのシャツと黒いズボンを着た男が、楽器を持っている人形を操っています。,男は人形と楽器を使って仕事をしています。
548012,contradiction,口ひげ、フェイスペイント、それに合った赤い帽子とシャツを着た男性がバイオリンを弾いています。,男は仕事を失った後、口ひげを剃ります。
548013,neutral,マヤ遺跡を訪れている間、ガイドはより多くのアトラクションの方向を指し示します。,人々はもっとやることを探している観光客です。


In [24]:
df.to_csv(corpus_path+new_name, index=False, encoding="UTF-8")